In [5]:
import os
import glob
import random
import pandas as pd
import datetime
import tiktoken
from collections import Counter, defaultdict
from IPython.display import display, Markdown

# user DF에서 History column과 Train column을 합쳐서 history로 사용
# file_name = "user16000_ns20_cd20.tsv"
file_name = "user16000_ns4_cd20.tsv"
# file_name = "user1000_ns20_cd20.tsv"
user = pd.read_csv(f'../../../data/LLM/{file_name}', sep='\t', names=['User', 'History', 'Question'])

NNR_news = pd.read_csv('../../../data/baseline/total_news(raw).tsv', sep='\t', names=['News ID', 'Category', 'SubCategory', 'Title', 'Body', 'ID', 'Publish', 'Click time history'], parse_dates=['Publish'])

# 데이터 확인

In [6]:
display(Markdown("### 1. user<hr/>"))
# display(user[user['User']==1]['Question'][0])
display(user)
display(Markdown("### 2. NNR_news<hr/>"))
display(NNR_news)

### 1. user<hr/>

,User,History,Question
0,1,"N2,2017-01-15 08:00:03;N2445,2017-01-16 00:17:...",N22972 N22963 N22975 N22962 N22964
1,1,"N2,2017-01-15 08:00:03;N2445,2017-01-16 00:17:...",N23283 N23263 N23262 N23269 N23260
2,1,"N2,2017-01-15 08:00:03;N2445,2017-01-16 00:17:...",N24116 N24110 N24115 N24195 N24293
3,1,"N2,2017-01-15 08:00:03;N2445,2017-01-16 00:17:...",N24109 N24103 N24108 N24260 N24098
4,1,"N2,2017-01-15 08:00:03;N2445,2017-01-16 00:17:...",N23981 N24074 N23974 N23968 N23975
...,...,...,...
145897,16000,"N11,2017-01-15 17:44:15;N5805,2017-01-17 04:56...",N23056 N23041 N23046 N23051 N23048
145898,16000,"N11,2017-01-15 17:44:15;N5805,2017-01-17 04:56...",N23173 N23392 N23156 N23154 N23151
145899,16000,"N11,2017-01-15 17:44:15;N5805,2017-01-17 04:56...",N23034 N23035 N23036 N23037 N23049
145900,16000,"N11,2017-01-15 17:44:15;N5805,2017-01-17 04:56...",N23204 N23205 N23624 N23203 N23389


### 2. NNR_news<hr/>

,News ID,Category,SubCategory,Title,Body,ID,Publish,Click time history
0,N1,100sport,sprek,Disse fire øvelsene bør du gjøre for å holde d...,Saken oppdateres. Dette mener hun er øvelsen m...,138920074713892198a921de316745902deece63,2017-01-15 07:27:55,"2017-02-13 17:18:27,2017-02-15 03:44:30,2017-0..."
1,N2,100sport,ballsport,Gravid i femte måned - vant NM-finalen,Saken oppdateres. Oslo Volley-Randaberg 3–0 Hu...,56b6283b3238ec631be66f4c120290ac666bdc4a,2017-01-14 14:26:01,"2017-02-14 01:37:10,2017-02-15 22:32:50,2017-0..."
2,N3,100sport,fotball,Afrikamesterskapet: Jubelen stilnet for Dortmu...,Saken oppdateres. Gruppe A: Gabon - Guinea-Bis...,ed0791368e611a277a94ff79e4445bf8a5bafa1b,2017-01-14 22:42:55,"2017-01-15 08:00:03,2017-01-15 08:02:04,2017-0..."
3,N4,100sport,ballsport,Berge om VM-debutanten: - Mann mot mann er han...,Saken oppdateres. NANTES: Den tidligere Viking...,cdf5d6552318de6f53f8b8ea2c105cff7204273b,2017-01-13 06:00:35,2017-02-08 19:13:51
4,N5,nyheter,innenriks,Norsk kvinne skadd i scooterulykke i Sverige,"Saken oppdateres. Åre grenser til Norge, øst f...",58e039f7560b9df8d3b34f977f64213ab2fa3aaa,2017-01-14 17:07:55,"2017-01-15 08:00:05,2017-01-15 08:32:59,2017-0..."
...,...,...,...,...,...,...,...,...
24055,N24436,nyheter,okonomi,29 streikende må tilbake på jobb,Saken oppdateres. Fellesforbundet avdeling 250...,7e6ad5f7078bf76fe7c9a5153c7dba72ee6d85b1,2016-04-28 05:38:17,2017-02-19 07:52:48
24056,N24437,nyheter,sortrondelag,To personer omkom etter at bil havnet i sjøen ...,Saken oppdateres. To personer er bekreftet omk...,15a6e6506a9e24f4bcdb9c31b7bf839c5c34d6cf,2013-10-25 13:00:06,2017-02-19 07:52:59
24057,N24438,nyheter,sortrondelag,Fant ingen tekniske feil på ulykkesbilen,Saken oppdateres. 25. oktober omkom to kvinner...,2a467a3af06df9de85059d27d8eb7602fce42fe6,2013-11-14 11:32:15,"2017-02-19 07:54:40,2017-02-19 07:58:06"
24058,N24439,nyheter,sortrondelag,Storåsfestivalen er konkurs,Saken oppdateres. Etter flere uker med hard me...,05f53546801c30193fe7b29e2264ac1b11eed6fd,2009-10-15 19:30:22,2017-02-19 07:55:40


# 코드 실행

In [7]:
import os
import glob
import random
import pandas as pd
import datetime
import tiktoken
from collections import Counter, defaultdict
from IPython.display import display, Markdown

#########################################################
# 1) Helper: extract_news_ids_and_click_times
#########################################################
def extract_news_ids_and_click_times(history_str):
    news_ids = []
    click_times = []
    seen_ids = set()

    entries = history_str.strip(';').split(';')
    for entry in entries:
        if not entry.strip():
            continue
        parts = entry.split(',')
        if len(parts) >= 2:
            news_id = parts[0].strip()
            click_time_str = parts[1].strip()
            click_time = pd.to_datetime(click_time_str, errors='coerce')
            if (news_id not in seen_ids) and pd.notnull(click_time):
                seen_ids.add(news_id)
                news_ids.append(news_id)
                click_times.append(click_time)
    return news_ids, click_times


#########################################################
# 2) Negative Sampling
#########################################################
def find_similar_news(
    news_df,
    news_ids,
    click_times,
    usage_id_counts,
    usage_title_counts,
    low_freq_categories,
    num_negatives=4
):
    """
    (1) 24시간 이내 + 부족시 더 과거
    (2) low_freq_categories 필터
    (3) 이미 사용된(news_id/title)이 limit_used(=2)회 이상이면 제외
    (4) 최대 4개 negative
    """
    all_neg_ids = []
    all_neg_titles = []
    limit_used = 2  # 예시: 2회까지 재사용 허용

    for h_id, ctime in zip(news_ids, click_times):
        time_window_start = ctime - pd.Timedelta(hours=24)

        # ----- 후보 1: 24시간 이내 ------
        candidate_news = news_df[
            (news_df['Publish'] >= time_window_start) &
            (news_df['Publish'] <= ctime) &
            (news_df['SubCategory'].isin(low_freq_categories))
        ].drop_duplicates(subset='Title')

        def is_usable(row):
            nid = row['News ID']
            ttl = row['Title']
            if usage_id_counts[nid] >= limit_used:
                return False
            if usage_title_counts[ttl] >= limit_used:
                return False
            return True

        candidate_rows = candidate_news.apply(is_usable, axis=1)
        candidate_news = candidate_news[candidate_rows]

        needed = num_negatives
        selected_list = []

        if len(candidate_news) >= needed:
            sel = candidate_news.sample(n=needed, random_state=42)
            selected_list.append(sel)
        else:
            selected_list.append(candidate_news)
            needed -= len(candidate_news)

            # ----- 후보 2: 24시간 이전(더 과거) -----
            extra_news = news_df[
                (news_df['Publish'] < time_window_start) &
                (news_df['SubCategory'].isin(low_freq_categories))
            ].drop_duplicates(subset='Title')

            extra_rows = extra_news.apply(is_usable, axis=1)
            extra_news = extra_news[extra_rows]

            extra_news = extra_news.assign(
                TimeDiff=(time_window_start - extra_news['Publish']).abs()
            )
            extra_news = extra_news.sort_values(by='TimeDiff')
            sel2 = extra_news.head(needed)
            selected_list.append(sel2)

        combined = pd.concat(selected_list)
        combined = combined.drop_duplicates(subset='Title').sample(frac=1, random_state=42)
        combined = combined.head(num_negatives)

        neg_ids = combined['News ID'].tolist()
        neg_titles = combined['Title'].tolist()

        for nid, ttl in zip(neg_ids, neg_titles):
            usage_id_counts[nid] += 1
            usage_title_counts[ttl] += 1

        all_neg_ids.append(neg_ids)
        all_neg_titles.append(neg_titles)

    return all_neg_ids, all_neg_titles


#########################################################
# 3) Demo generator (old vs. new)
#########################################################
def generate_demonstrations_old_style(
    user_id,
    history_ids,
    history_times,
    news_df,
    usage_id_counts,
    usage_title_counts,
    category_use=False,
    purpose="with_negative"
):
    """
    If purpose=='only_positive', just return lines with the user's actual clicked titles (no negative).
    Otherwise (with_negative), produce 5-article blocks (4 negatives + 1 positive).
    Returns: lines, history_titles, history_neg_titles
    """
    history_titles = []
    for hid in history_ids:
        row_ = news_df[news_df['News ID'] == hid]
        if row_.empty:
            history_titles.append(f"(Unknown Title for {hid})")
        else:
            history_titles.append(row_.iloc[0]['Title'])

    if purpose == "only_positive":
        # Just output the positive clicked history
        lines = []
        lines.append("[Click History]")
        lines.append(f"The news articles that User #{user_id} clicked before are as follows:\n")
        for idx, (hid, htitle) in enumerate(zip(history_ids, history_titles), start=1):
            row_ = news_df[news_df['News ID'] == hid]
            if not row_.empty:
                subcat = row_.iloc[0]['SubCategory']
            else:
                subcat = "unknown"

            if category_use:
                lines.append(f"{idx}. {htitle} [category : {subcat}]")
            else:
                lines.append(f"{idx}. {htitle}")
        lines.append("")  # blank line

        # No negative
        history_neg_titles = [[] for _ in history_ids]
        return lines, history_titles, history_neg_titles

    else:
        # with_negative
        limit_count = 2
        subcats = []
        for hid in history_ids:
            row_ = news_df[news_df['News ID'] == hid]
            if not row_.empty:
                subcats.append(row_.iloc[0]['SubCategory'])
            else:
                subcats.append(None)

        counts = Counter(subcats)
        all_subcats = news_df['SubCategory'].dropna().unique()
        low_freq_categories = [cat for cat in all_subcats if counts[cat] < limit_count]

        # negative sampling
        history_neg_ids, history_neg_titles = find_similar_news(
            news_df=news_df,
            news_ids=history_ids,
            click_times=history_times,
            usage_id_counts=usage_id_counts,
            usage_title_counts=usage_title_counts,
            low_freq_categories=low_freq_categories,
            num_negatives=4
        )

        lines = ["[News of Interest to the user]\n"]
        for idx, (htitle, neg_list) in enumerate(zip(history_titles, history_neg_titles), start=1):
            row_cat_htitle = news_df[news_df['Title'] == htitle]
            if not row_cat_htitle.empty:
                htitle_subcat = row_cat_htitle.iloc[0]['SubCategory']
            else:
                htitle_subcat = "unknown"

            combined_titles = neg_list + [htitle]
            random.shuffle(combined_titles)

            if category_use:
                lines.append(
                    f"{idx}) User #{user_id} prefers most {htitle} [category : {htitle_subcat}] "
                    "among the following five articles:"
                )
            else:
                lines.append(
                    f"{idx}) User #{user_id} prefers most {htitle} among the following five articles:"
                )

            for i, cand_title in enumerate(combined_titles, start=1):
                row_cat = news_df[news_df['Title'] == cand_title]
                if not row_cat.empty:
                    subcat = row_cat.iloc[0]['SubCategory']
                else:
                    subcat = "unknown"

                if category_use:
                    lines.append(f"{i}: {cand_title} [category : {subcat}]")
                else:
                    lines.append(f"{i}: {cand_title}")
            lines.append("")

        return lines, history_titles, history_neg_titles


#########################################################
# 3-b) Demo generator for "origin" method
#########################################################
def generate_demonstrations_origin(
    user_id,
    history_ids,
    history_times,
    news_df,
    usage_id_counts,
    usage_title_counts,
    category_use=False
):
    """
    Similar to 'with_negative', but we produce a different text format:
      1. All 5 articles in a single line, joined by ' / '
      2. Then a line: "Of the five news above, ... <clicked article>"

    Returns:
      lines, history_titles, history_neg_titles
    """
    history_titles = []
    for hid in history_ids:
        row_ = news_df[news_df['News ID'] == hid]
        if row_.empty:
            history_titles.append(f"(Unknown Title for {hid})")
        else:
            history_titles.append(row_.iloc[0]['Title'])

    # We'll do negative sampling exactly as in 'with_negative'
    limit_count = 2
    subcats = []
    for hid in history_ids:
        row_ = news_df[news_df['News ID'] == hid]
        if not row_.empty:
            subcats.append(row_.iloc[0]['SubCategory'])
        else:
            subcats.append(None)

    counts = Counter(subcats)
    all_subcats = news_df['SubCategory'].dropna().unique()
    low_freq_categories = [cat for cat in all_subcats if counts[cat] < limit_count]

    # negative sampling
    history_neg_ids, history_neg_titles = find_similar_news(
        news_df=news_df,
        news_ids=history_ids,
        click_times=history_times,
        usage_id_counts=usage_id_counts,
        usage_title_counts=usage_title_counts,
        low_freq_categories=low_freq_categories,
        num_negatives=4
    )

    # Build lines in the new "origin" style
    lines = ["[News of Interest to the user]\n"]

    for idx, (htitle, neg_list) in enumerate(zip(history_titles, history_neg_titles), start=1):
        # find category for the clicked item
        row_cat_htitle = news_df[news_df['Title'] == htitle]
        if not row_cat_htitle.empty:
            htitle_subcat = row_cat_htitle.iloc[0]['SubCategory']
        else:
            htitle_subcat = "unknown"

        # combine
        combined_titles = neg_list + [htitle]
        random.shuffle(combined_titles)

        # make a single line of "1. Title [category: x] / 2. Title [category: y] / ..."
        items_str_list = []
        for i, cand_title in enumerate(combined_titles, start=1):
            row_cat = news_df[news_df['Title'] == cand_title]
            if not row_cat.empty:
                subcat = row_cat.iloc[0]['SubCategory']
            else:
                subcat = "unknown"

            if category_use:
                items_str_list.append(f"{cand_title} [category : {subcat}]")
            else:
                items_str_list.append(f"{cand_title}")

            # if category_use:
            #     items_str_list.append(f"{i}. {cand_title} [category : {subcat}]")
            # else:
            #     items_str_list.append(f"{i}. {cand_title}")
            

        line_of_5 = " / ".join(items_str_list)

        lines.append(f"{idx}. {line_of_5}")

        # Then the line stating which one was actually clicked
        if category_use:
            lines.append(
                "Of the five news above, the news that the user is most interested in : "
                f"{htitle} [category : {htitle_subcat}]"
            )
        else:
            lines.append(
                "Of the five news above, the news that the user is most interested in : "
                f"{htitle}"
            )

        lines.append("")  # blank line

    return lines, history_titles, history_neg_titles


#########################################################
# 4) Metadata
#########################################################
def save_metadata_file(meta_folder_path, token_stats_list):
    if os.path.exists(meta_folder_path):
        for file_path in glob.glob(os.path.join(meta_folder_path, "*.txt")):
            os.remove(file_path)
        print(f"[INFO] Metadata folder cleared: {meta_folder_path}")

    os.makedirs(meta_folder_path, exist_ok=True)

    meta_file_path = os.path.join(meta_folder_path, "output_metadata.txt")

    total_input_tokens = 0
    total_output_count = 0

    # Sort by user ID numerically if possible
    def try_int(x):
        try:
            return int(x)
        except:
            return x

    sorted_list = sorted(token_stats_list, key=lambda x: try_int(x[0]))

    with open(meta_file_path, "w", encoding="utf-8") as m:
        for (uid, tcount, oc, hnum, qnum) in sorted_list:
            line = (f"User ID: U{uid:<5}  "
                    f"Input Tokens: {tcount:<6}  "
                    f"Output Tokens: {oc:<4}  "
                    f"History 수: {hnum:<3}  "
                    f"Question 수: {qnum}")
            total_input_tokens += tcount
            total_output_count += oc
            m.write(line + "\n")

        sum_line = (f"\nTotal Input Tokens: {total_input_tokens}\n"
                    f"Total Output Tokens: {total_output_count}\n")
        m.write(sum_line)
        print("[INFO]", sum_line)


def save_hidden_positions(meta_folder_path, hidden_positions_data):
    hidden_path = os.path.join(meta_folder_path, "hidden_positions.txt")
    with open(hidden_path, "w", encoding="utf-8") as f:
        for uid, positions in hidden_positions_data:
            f.write(f"U{uid:<5}: {positions}\n")


#########################################################
# 5) Main
#########################################################
def create_prompts_new_df_with_clicktime(
    user_df,
    news_df,
    output_folder="prompt",
    model_name="gpt-4",
    max_history=None,
    max_question=None,
    start_user=1,
    max_user=None,
    category_use=False,
    purpose="with_negative",
    method="ranking"
):
    """
    purpose:
      - 'with_negative' : original negative-sampling approach
      - 'only_positive' : no negative sampling, only user-clicked titles
    method:
      - 'ranking'       : ask user to arrange top 5
      - 'top1'          : ask user to pick best 1 among N
      - 'origin'        : produce the new text format as described
    """
    encoding = tiktoken.encoding_for_model(model_name)
    output_folder = f'../../prompts/{output_folder}'

    start_user -= 1

    # Filter user_df by max_user if needed
    if max_user is not None:
        unique_users = sorted(user_df["User"].unique(), key=lambda x: int(x))
        selected_users = set(unique_users[start_user:max_user])
        user_df = user_df[user_df["User"].isin(selected_users)]

    # Build target folder
    target_folder = os.path.join(output_folder, purpose)
    if os.path.exists(target_folder):
        for f in glob.glob(os.path.join(target_folder, "*.txt")):
            os.remove(f)
        print(f"[INFO] Existing prompts removed: {target_folder}")
    else:
        os.makedirs(target_folder, exist_ok=True)

    token_stats_list = []
    hidden_positions_data = []

    grouped = user_df.groupby("User")
    user_prompts = {}

    for user_val, group in grouped:
        user_id = str(user_val)

        usage_id_counts = defaultdict(int)
        usage_title_counts = defaultdict(int)

        # Parse history
        first_row = group.iloc[0]
        history_str = first_row["History"]
        all_history_ids, all_history_times = extract_news_ids_and_click_times(history_str)

        # max_history
        if max_history is not None:
            pairs = sorted(zip(all_history_ids, all_history_times), key=lambda x: x[1])
            pairs = pairs[-max_history:]
            history_ids = [p[0] for p in pairs]
            history_times = [p[1] for p in pairs]
        else:
            history_ids = all_history_ids
            history_times = all_history_times

        # Depending on method=="origin" or not, we generate the demonstration lines
        if method == "origin":
            # We'll produce negative sampling but format it differently
            demo_lines, history_titles, history_neg_titles = generate_demonstrations_origin(
                user_id=user_id,
                history_ids=history_ids,
                history_times=history_times,
                news_df=news_df,
                usage_id_counts=usage_id_counts,
                usage_title_counts=usage_title_counts,
                category_use=category_use
            )
        else:
            # Use the old logic ('with_negative' or 'only_positive')
            demo_lines, history_titles, history_neg_titles = generate_demonstrations_old_style(
                user_id=user_id,
                history_ids=history_ids,
                history_times=history_times,
                news_df=news_df,
                usage_id_counts=usage_id_counts,
                usage_title_counts=usage_title_counts,
                category_use=category_use,
                purpose=purpose
            )

        # ============================================================
        # [Questions] part (use the method to decide prompt style)
        # ============================================================
        row_records = group.to_dict("records")
        if max_question is not None:
            row_records = row_records[:max_question]

        local_question_count = 0
        hidden_positions = []

        # We build question lines based on method
        if method == "ranking":
            question_header = (
                f"[Questions]\nBased on User #{user_id}'s preferences, "
                f"arrange the index numbers of the top five news articles in the sequence "
                f"that is deemed most suitable for [MASK].\n"
            )
            question_footer = (
                f"Please provide just the answers to each of User #{user_id}'s question "
                f"without any explanations."
            )
        elif method == "top1":
            question_header = (
                f"[Questions]\nBased on User #{user_id}'s preferences, "
                f"predict the index number of the news article that best fits "
                f"the position labeled [MASK] for each question.\n"
            )
            question_footer = (
                f"Please provide just the answers to each of User #{user_id}'s question "
                f"without any explanations."
            )
        elif method == "origin":
            # "origin" style question text
            question_header = (
                "[Questions]\n"
                "Returns the most likely clickable news among the candidate news "
                "for each question based on the user's news interests.\n"
            )
            question_footer = "Don't explain why in your answer, just return the number of the news."
        else:
            # fallback
            question_header = "[Questions]\n"
            question_footer = "Please provide just the answers."

        question_lines = [question_header]

        for idx_q, row_data in enumerate(row_records, start=1):
            q_str = row_data["Question"]
            q_ids = q_str.split()
            if not q_ids:
                continue

            pos_id = q_ids[0]
            row_m = news_df[news_df["News ID"] == pos_id]
            if row_m.empty:
                pos_title = f"(Unknown Title for {pos_id})"
                pos_subcat = "unknown"
            else:
                pos_title = row_m.iloc[0]["Title"]
                pos_subcat = row_m.iloc[0]["SubCategory"]

            # build candidate list
            candidate_info = []
            for qid in q_ids:
                mm = news_df[news_df["News ID"] == qid]
                if mm.empty:
                    candidate_info.append(("(Unknown Title for %s)" % qid, "unknown"))
                else:
                    ctitle = mm.iloc[0]["Title"]
                    csubcat = mm.iloc[0]["SubCategory"]
                    candidate_info.append((ctitle, csubcat))

            random.shuffle(candidate_info)
            # figure out hidden answer's index
            pos_index = -1
            for i_c, (ctitle, csubcat) in enumerate(candidate_info, start=1):
                if ctitle == pos_title:
                    pos_index = i_c
                    break
            hidden_positions.append(pos_index)

            if method == "origin":
                # Single-line enumerations with slash
                # e.g. "Question 1) 1: Title [cat] / 2: Title [cat] / ...
                cand_str_list = []
                for i_c, (ctitle, csubcat) in enumerate(candidate_info, start=1):
                    if category_use:
                        cand_str_list.append(f"{i_c}: {ctitle} [category : {csubcat}]")
                    else:
                        cand_str_list.append(f"{i_c}: {ctitle}")
                line_of_candidates = " / ".join(cand_str_list)

                question_lines.append(f"Question {idx_q}) {line_of_candidates}")
                question_lines.append("")  # blank
            else:
                # Original method for ranking/top1
                if method == "ranking":
                    question_lines.append(
                        f"Question {idx_q}) User #{user_id} prefers most [MASK] among the following {len(candidate_info)} articles:"
                    )
                elif method == "top1":
                    question_lines.append(
                        f"Question {idx_q}) User #{user_id} prefers most [MASK] among the following {len(candidate_info)} articles:"
                    )
                else:
                    question_lines.append(f"Question {idx_q})")

                for i_c, (ctitle, csubcat) in enumerate(candidate_info, start=1):
                    if category_use:
                        question_lines.append(f"{i_c}: {ctitle} [category : {csubcat}]")
                    else:
                        question_lines.append(f"{i_c}: {ctitle}")
                question_lines.append("")

            local_question_count += 1

        question_lines.append(question_footer)

        # Combine
        final_prompt = "\n".join(demo_lines + question_lines)
        user_prompts[user_id] = final_prompt

        # Token count
        token_count = len(encoding.encode(final_prompt))
        output_count = local_question_count
        num_history = len(history_titles)

        token_stats_list.append((user_id, token_count, output_count, num_history, local_question_count))
        hidden_positions_data.append((user_id, hidden_positions))

    # Save out
    for uid, prompt_txt in user_prompts.items():
        out_path = os.path.join(target_folder, f"U{uid}.txt")
        with open(out_path, "w", encoding="utf-8") as ff:
            ff.write(prompt_txt)

    print(f"[INFO] All prompts created in: {target_folder}")

    # metadata
    meta_folder_path = os.path.join(target_folder, "metadata")
    save_metadata_file(meta_folder_path, token_stats_list)
    save_hidden_positions(meta_folder_path, hidden_positions_data)
    print("[INFO] Metadata saved.")


## 실행
[create 함수]
- purpose = 어떤 목적으로 prompt를 생성할 것인지  [only_positive / with_negative]   
- model = 사용할 gpt (token 계산 용도)  [gpt-4o-mini / gpt-3.5-turbo]
- user_count = 몇 명의 user prompt를 생성할 것인지
- max_question = 최대 질문 수
- save_forder = 결과를 저장할 폴더 이름
- max_history: History에서 최근 n개만
- max_question: Question에서 n개 행만
- max_user: 사용자 상위 n명만 처리 (User 값이 숫자라고 가정)
- method : top1 / ranking / origin 

In [8]:
# 1) 목적(purpose) == 'with_negative' (neg sampling 사용)
#    질문(method) == 'ranking' (상위 5개 뉴스 순위)
create_prompts_new_df_with_clicktime(
    user_df=user,
    news_df=NNR_news,
    output_folder="[top1] test_ns4",
    model_name="gpt-4",
    max_history=None,
    max_question=None,
    start_user=1,
    max_user=1000,
    category_use=True,
    purpose="with_negative",   # 여기서 'with_negative'
    method="top1"
)

# # 2) only_positive (네거티브 샘플링 없이)
# create_prompts_new_df_with_clicktime(
#     user_df=user,
#     news_df=NNR_news,
#     output_folder="[250204]",
#     model_name="gpt-4",
#     max_history=None,
#     max_question=None,
#     start_user=1,
#     max_user=1000,
#     category_use=True,
#     purpose="only_positive",   # 여기서 'only_positive'
#     method="top1"
# )


[INFO] All prompts created in: ../../prompts/[top1] test_ns4\with_negative
[INFO] 
Total Input Tokens: 9991734
Total Output Tokens: 9307

[INFO] Metadata saved.


## ---

In [10]:
def get_category_counts(user_df: pd.DataFrame, news_df: pd.DataFrame, user_id: int):
    """
    특정 사용자의 History에 포함된 뉴스들의 Category 및 SubCategory 개수를 계산하여 출력.
    
    Parameters:
    - user_df (pd.DataFrame): 사용자 데이터프레임
    - news_df (pd.DataFrame): 뉴스 데이터프레임
    - user_id (int): 조회할 사용자 ID
    
    Returns:
    - category_counts (pd.Series): Category별 개수 (내림차순 정렬)
    - subcategory_counts (pd.Series): SubCategory별 개수 (내림차순 정렬)
    """
    # 특정 User의 History 가져오기
    user_history_series = user_df.loc[user_df["User"] == user_id, "History"]
    
    # 해당 유저의 기록이 없으면 예외 처리
    if user_history_series.empty:
        print(f"User {user_id}에 대한 데이터가 없습니다.")
        return None, None

    user_history = user_history_series.values[0]  # 같은 값이므로 첫 번째 값만 가져옴
    
    # 뉴스 ID 추출
    news_ids = [entry.split(",")[0] for entry in user_history.split(";")]

    # news_df에서 해당 뉴스 ID 필터링
    filtered_news = news_df[news_df["News ID"].isin(news_ids)]

    # Category 및 SubCategory 개수 집계 및 정렬
    category_counts = filtered_news["Category"].value_counts()
    subcategory_counts = filtered_news["SubCategory"].value_counts()

    return category_counts, subcategory_counts


In [11]:
# 예제 실행
category_counts, subcategory_counts = get_category_counts(user, NNR_news, 1000)

# 결과 출력
print("Category 개수:\n", category_counts)
print("\nSubCategory 개수:\n", subcategory_counts)

Category 개수:
 Category
nyheter     32
100sport    15
pluss        5
meninger     1
Name: count, dtype: int64

SubCategory 개수:
 SubCategory
sortrondelag     14
trondheim        10
vintersport       6
fotball           6
nordtrondelag     5
okonomi           3
magasin           2
sprek             1
ballsport         1
andreidretter     1
utenriks          1
kultur            1
kronikker         1
nyheter           1
Name: count, dtype: int64
